In [28]:
import os
import shutil

# Path to the folder
folder_path = '/content/sample_data/Image'

# Iterate over all files in the folder and delete them
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        # If you want to delete subdirectories as well, uncomment the line below
        # elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")


In [1]:
import cv2
import os
import glob
import numpy as np
import mediapipe as mp


## Without Using Face Landmarks

In [2]:
# Initialize MediaPipe face detection and face mesh modules
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh

# Function to extract left and right eyes from frames using MediaPipe
def extract_eyes_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names):
    # Create output directories if they don't exist
    os.makedirs(left_eye_output_dir, exist_ok=True)
    os.makedirs(right_eye_output_dir, exist_ok=True)

    # Initialize MediaPipe face detection
    with mp_face_detection.FaceDetection(min_detection_confidence=0.7) as face_detection:
        for frame, frame_name in zip(frames, frame_names):
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect faces in the frame
            results = face_detection.process(frame_rgb)

            if results.detections:
                for detection in results.detections:
                    # Extract bounding box coordinates
                    bbox = detection.location_data.relative_bounding_box
                    h, w, _ = frame.shape
                    x, y, width, height = int(bbox.xmin * w), int(bbox.ymin * h), int(bbox.width * w), int(bbox.height * h)

                    # Define eye regions based on bounding box
                    eye_x = x + int(width * 0.15)
                    eye_y = y + int(height * 0.2)
                    eye_width = int(width * 0.35)
                    eye_height = int(height * 0.4)

                    # Crop and flip left eye region
                    left_eye = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]
                    if left_eye.size == 0:
                        print(f"Left eye image is empty for frame {frame_name}")
                    else:
                        left_eye_resized = cv2.resize(left_eye, (224, 224)) # Resize to required size for EfficientNet
                    left_eye_gray = cv2.cvtColor(left_eye_resized, cv2.COLOR_BGR2GRAY)
                    left_eye_flipped = cv2.flip(left_eye_gray, 1)  # Flip the left eye image horizontally
                    left_eye_filename = os.path.join(left_eye_output_dir, frame_name)
                    cv2.imwrite(left_eye_filename, left_eye_flipped)

                    # Crop right eye region
                    right_eye_x = x + int(width * 0.5)
                    right_eye = frame[eye_y:eye_y+eye_height, right_eye_x:right_eye_x+eye_width]
                    if right_eye.size == 0:
                        print(f"Right eye image is empty for frame {frame_name}")
                    else:
                        right_eye_resized = cv2.resize(right_eye, (224, 224))  # Resize to required size for EfficientNet
                    right_eye_gray = cv2.cvtColor(right_eye_resized, cv2.COLOR_BGR2GRAY)
                    right_eye_filename = os.path.join(right_eye_output_dir, frame_name)
                    cv2.imwrite(right_eye_filename, right_eye_gray)

    print("Eye extraction complete.")

# Function to process directories and extract eyes
def process_directories(input_dir):
    main_output_dir = "New Extract Eye With Time"
    os.makedirs(main_output_dir, exist_ok=True)

    subdirectories = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(input_dir, subdirectory)
        image_files = glob.glob(os.path.join(subdirectory_path, "*.jpg"))
        
        frames = [cv2.imread(img) for img in image_files]
        frame_names = [os.path.basename(img) for img in image_files]

        left_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Left Eye")
        right_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Right Eye")

        extract_eyes_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names)

# Example usage:
input_directory = "D:\\harsh\\Blepharospasm\\New Extracted Frames"
process_directories(input_directory)


c:\Users\harsh\anaconda3\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Eye extraction complete.
Eye extraction complete.
Left eye image is empty for frame frame_10050ms.jpg
Left eye image is empty for frame frame_10084ms.jpg
Left eye image is empty for frame frame_10117ms.jpg
Left eye image is empty for frame frame_10150ms.jpg
Left eye image is empty for frame frame_10350ms.jpg
Left eye image is empty for frame frame_10384ms.jpg
Left eye image is empty for frame frame_10617ms.jpg
Left eye image is empty for frame frame_10650ms.jpg
Left eye image is empty for frame frame_10684ms.jpg
Left eye image is empty for frame frame_10717ms.jpg
Left eye image is empty for frame frame_10750ms.jpg
Left eye image is empty for frame frame_10784ms.jpg
Left eye image is empty for frame frame_10817ms.jpg
Left eye image is empty for frame frame_10917ms.jpg
Left eye image is empty for frame frame_11017ms.jpg
Left eye image is empty for frame frame_11217ms.jpg
Left eye image is empty for frame frame_11284ms.jpg
Left eye image is empty for frame frame_11317ms.jpg
Left eye image

## Using Face Landmarks

In [1]:
import cv2
import os
import glob
import mediapipe as mp
import numpy as np

In [2]:


# Initialize MediaPipe face mesh module
mp_face_mesh = mp.solutions.face_mesh

# Indices for left and right eye landmarks based on MediaPipe's face mesh model
RIGHT_EYE_INDICES = [33, 133, 160, 159, 158, 157, 173, 155, 154, 153, 144, 145, 153, 163, 7, 33]
LEFT_EYE_INDICES = [263, 362, 387, 386, 385, 384, 398, 382, 381, 380, 373, 374, 380, 390, 249, 263]

# Function to extract left and right eyes from frames using MediaPipe
def extract_eyes_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names, padding=22):
    # Create output directories if they don't exist
    os.makedirs(left_eye_output_dir, exist_ok=True)
    os.makedirs(right_eye_output_dir, exist_ok=True)

    # Initialize MediaPipe face mesh
    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.7) as face_mesh:
        for frame, frame_name in zip(frames, frame_names):
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process frame to get face mesh
            mesh_results = face_mesh.process(frame_rgb)

            if mesh_results.multi_face_landmarks:
                for face_landmarks in mesh_results.multi_face_landmarks:
                    # Extract coordinates for left and right eye landmarks
                    left_eye_landmarks = [face_landmarks.landmark[i] for i in LEFT_EYE_INDICES]
                    right_eye_landmarks = [face_landmarks.landmark[i] for i in RIGHT_EYE_INDICES]

                    # Convert normalized landmarks to pixel coordinates
                    def landmarks_to_points(landmarks):
                        return [(int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])) for lm in landmarks]

                    left_eye_points = landmarks_to_points(left_eye_landmarks)
                    right_eye_points = landmarks_to_points(right_eye_landmarks)

                    # Function to extract the region defined by a set of points
                    def crop_eye_region(frame, points, padding=20):
                        x_coords = [p[0] for p in points]
                        y_coords = [p[1] for p in points]

                        # Bounding box around the eyes with padding
                        x_min, x_max = max(0, min(x_coords) - padding), min(frame.shape[1], max(x_coords) + padding)
                        y_min, y_max = max(0, min(y_coords) - padding), min(frame.shape[0], max(y_coords) + padding)

                        return frame[y_min:y_max, x_min:x_max]

                    # Crop, resize, and convert left eye region to grayscale
                    left_eye = crop_eye_region(frame, left_eye_points, padding)
                    if left_eye.size != 0:
                        left_eye_resized = cv2.resize(left_eye, (224, 224))
                        left_eye_gray = cv2.cvtColor(left_eye_resized, cv2.COLOR_BGR2GRAY)
                        left_eye_flipped = cv2.flip(left_eye_gray, 1)  # Flip the left eye image horizontally
                        left_eye_filename = os.path.join(left_eye_output_dir, frame_name)
                        cv2.imwrite(left_eye_filename, left_eye_flipped)
                    else: print(f"Left eye image is empty for frame {frame_name}")

                    # Crop, resize, and convert right eye region to grayscale
                    right_eye = crop_eye_region(frame, right_eye_points, padding)
                    if right_eye.size != 0:
                        right_eye_resized = cv2.resize(right_eye, (224, 224))
                        right_eye_gray = cv2.cvtColor(right_eye_resized, cv2.COLOR_BGR2GRAY)
                        #right_eye_flipped = cv2.flip(right_eye_gray, 1)  # Flip the left eye image horizontally
                        right_eye_filename = os.path.join(right_eye_output_dir, frame_name)
                        cv2.imwrite(right_eye_filename, right_eye_gray)
                    else: print(f"Right eye image is empty for frame {frame_name}")    

    print("Eye extraction complete.")

# Function to process directories and extract eyes
def process_directories(input_dir):
    main_output_dir = "D:\\harsh\\Blepharospasm\\env\\Normal Eyes With Time"
    os.makedirs(main_output_dir, exist_ok=True)

    subdirectories = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(input_dir, subdirectory)
        image_files = glob.glob(os.path.join(subdirectory_path, "*.jpg"))
        
        frames = [cv2.imread(img) for img in image_files]
        frame_names = [os.path.basename(img) for img in image_files]

        left_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Left Eye")
        right_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Right Eye")

        extract_eyes_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names)

# Example usage:
input_directory = "D:\\harsh\\Blepharospasm\\Extracted Colored Eye\\New Extracted Frames"
process_directories(input_directory)


c:\Users\harsh\anaconda3\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Eye extraction complete.


In [4]:
import cv2
import mediapipe as mp
import os
import glob

# Initialize MediaPipe face mesh module
mp_face_mesh = mp.solutions.face_mesh

# Indices for left and right eye landmarks based on MediaPipe's face mesh model
RIGHT_EYE_INDICES = [33, 133, 160, 159, 158, 157, 173, 155, 154, 153, 144, 145, 153, 163, 7, 33]
LEFT_EYE_INDICES = [263, 362, 387, 386, 385, 384, 398, 382, 381, 380, 373, 374, 380, 390, 249, 263]

# Indices for left and right eyebrow landmarks based on MediaPipe's face mesh model
RIGHT_EYEBROW_INDICES = [70, 63, 105, 66, 107, 55, 193]
LEFT_EYEBROW_INDICES = [336, 296, 334, 293, 300, 283, 265]

# Function to extract eyes and eyebrows from frames using MediaPipe
def extract_eyes_eyebrows_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names, padding=22):
    # Create output directories if they don't exist
    os.makedirs(left_eye_output_dir, exist_ok=True)
    os.makedirs(right_eye_output_dir, exist_ok=True)

    # Initialize MediaPipe face mesh
    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.7) as face_mesh:
        for frame, frame_name in zip(frames, frame_names):
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process frame to get face mesh
            mesh_results = face_mesh.process(frame_rgb)

            if mesh_results.multi_face_landmarks:
                for face_landmarks in mesh_results.multi_face_landmarks:
                    # Extract coordinates for left and right eye landmarks
                    left_eye_landmarks = [face_landmarks.landmark[i] for i in LEFT_EYE_INDICES]
                    right_eye_landmarks = [face_landmarks.landmark[i] for i in RIGHT_EYE_INDICES]

                    # Extract coordinates for left and right eyebrow landmarks
                    left_eyebrow_landmarks = [face_landmarks.landmark[i] for i in LEFT_EYEBROW_INDICES]
                    right_eyebrow_landmarks = [face_landmarks.landmark[i] for i in RIGHT_EYEBROW_INDICES]

                    # Combine eye and eyebrow landmarks
                    left_eye_eyebrow_landmarks = left_eye_landmarks + left_eyebrow_landmarks
                    right_eye_eyebrow_landmarks = right_eye_landmarks + right_eyebrow_landmarks

                    # Convert normalized landmarks to pixel coordinates
                    def landmarks_to_points(landmarks):
                        return [(int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])) for lm in landmarks]

                    left_eye_eyebrow_points = landmarks_to_points(left_eye_eyebrow_landmarks)
                    right_eye_eyebrow_points = landmarks_to_points(right_eye_eyebrow_landmarks)

                    # Function to extract the region defined by a set of points
                    def crop_eye_eyebrow_region(frame, points, padding=20):
                        x_coords = [p[0] for p in points]
                        y_coords = [p[1] for p in points]

                        # Bounding box around the eyes and eyebrows with padding
                        x_min, x_max = max(0, min(x_coords) - padding), min(frame.shape[1], max(x_coords) + padding)
                        y_min, y_max = max(0, min(y_coords) - padding), min(frame.shape[0], max(y_coords) + padding)

                        return frame[y_min:y_max, x_min:x_max]

                    # Crop, resize, and convert left eye and eyebrow region to grayscale
                    left_eye_eyebrow = crop_eye_eyebrow_region(frame, left_eye_eyebrow_points, padding)
                    if left_eye_eyebrow.size != 0:
                        left_eye_eyebrow_resized = cv2.resize(left_eye_eyebrow, (224, 224))
                        left_eye_eyebrow_gray = cv2.cvtColor(left_eye_eyebrow_resized, cv2.COLOR_BGR2GRAY)
                        left_eye_eyebrow_flipped = cv2.flip(left_eye_eyebrow_gray, 1)  # Flip the left eye image horizontally
                        left_eye_filename = os.path.join(left_eye_output_dir, frame_name)
                        cv2.imwrite(left_eye_filename, left_eye_eyebrow_flipped)
                    else:
                        print(f"Left eye and eyebrow image is empty for frame {frame_name}")

                    # Crop, resize, and convert right eye and eyebrow region to grayscale
                    right_eye_eyebrow = crop_eye_eyebrow_region(frame, right_eye_eyebrow_points, padding)
                    if right_eye_eyebrow.size != 0:
                        right_eye_eyebrow_resized = cv2.resize(right_eye_eyebrow, (224, 224))
                        right_eye_eyebrow_gray = cv2.cvtColor(right_eye_eyebrow_resized, cv2.COLOR_BGR2GRAY)
                        right_eye_filename = os.path.join(right_eye_output_dir, frame_name)
                        cv2.imwrite(right_eye_filename, right_eye_eyebrow_gray)
                    else:
                        print(f"Right eye and eyebrow image is empty for frame {frame_name}")

    print("Eye and eyebrow extraction complete.")

# Function to process directories and extract eyes and eyebrows
def process_directories(input_dir):
    main_output_dir = "D:\\harsh\\Blepharospasm\\env\\Normal Eyes With Time"
    os.makedirs(main_output_dir, exist_ok=True)

    subdirectories = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(input_dir, subdirectory)
        image_files = glob.glob(os.path.join(subdirectory_path, "*.jpg"))
        
        frames = [cv2.imread(img) for img in image_files]
        frame_names = [os.path.basename(img) for img in image_files]

        left_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Left Eye")
        right_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Right Eye")

        extract_eyes_eyebrows_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names)

# Example usage:
input_directory = "D:\\harsh\\Blepharospasm\\Extracted Colored Eye\\New Extracted Frames"
process_directories(input_directory)


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 1200000 bytes in function 'cv::OutOfMemoryError'


# Extracted Colored Eye

In [2]:
import cv2
import mediapipe as mp
import os
import glob

# Initialize MediaPipe face mesh module
mp_face_mesh = mp.solutions.face_mesh

# Indices for left and right eye landmarks based on MediaPipe's face mesh model
RIGHT_EYE_INDICES = [33, 133, 160, 159, 158, 157, 173, 155, 154, 153, 144, 145, 153, 163, 7, 33]
LEFT_EYE_INDICES = [263, 362, 387, 386, 385, 384, 398, 382, 381, 380, 373, 374, 380, 390, 249, 263]

# Indices for left and right eyebrow landmarks based on MediaPipe's face mesh model
RIGHT_EYEBROW_INDICES = [70, 63, 105, 66, 107, 55, 193]
LEFT_EYEBROW_INDICES = [336, 296, 334, 293, 300, 283, 265]

# Function to extract eyes and eyebrows from frames using MediaPipe
def extract_eyes_eyebrows_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names, padding=22):
    # Create output directories if they don't exist
    os.makedirs(left_eye_output_dir, exist_ok=True)
    os.makedirs(right_eye_output_dir, exist_ok=True)

    # Initialize MediaPipe face mesh
    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.7) as face_mesh:
        for frame, frame_name in zip(frames, frame_names):
            # Convert frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process frame to get face mesh
            mesh_results = face_mesh.process(frame_rgb)

            if mesh_results.multi_face_landmarks:
                for face_landmarks in mesh_results.multi_face_landmarks:
                    # Extract coordinates for left and right eye landmarks
                    left_eye_landmarks = [face_landmarks.landmark[i] for i in LEFT_EYE_INDICES]
                    right_eye_landmarks = [face_landmarks.landmark[i] for i in RIGHT_EYE_INDICES]

                    # Extract coordinates for left and right eyebrow landmarks
                    left_eyebrow_landmarks = [face_landmarks.landmark[i] for i in LEFT_EYEBROW_INDICES]
                    right_eyebrow_landmarks = [face_landmarks.landmark[i] for i in RIGHT_EYEBROW_INDICES]

                    # Combine eye and eyebrow landmarks
                    left_eye_eyebrow_landmarks = left_eye_landmarks + left_eyebrow_landmarks
                    right_eye_eyebrow_landmarks = right_eye_landmarks + right_eyebrow_landmarks

                    # Convert normalized landmarks to pixel coordinates
                    def landmarks_to_points(landmarks):
                        return [(int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])) for lm in landmarks]

                    left_eye_eyebrow_points = landmarks_to_points(left_eye_eyebrow_landmarks)
                    right_eye_eyebrow_points = landmarks_to_points(right_eye_eyebrow_landmarks)

                    # Function to extract the region defined by a set of points
                    def crop_eye_eyebrow_region(frame, points, padding=20):
                        x_coords = [p[0] for p in points]
                        y_coords = [p[1] for p in points]

                        # Bounding box around the eyes and eyebrows with padding
                        x_min, x_max = max(0, min(x_coords) - padding), min(frame.shape[1], max(x_coords) + padding)
                        y_min, y_max = max(0, min(y_coords) - padding), min(frame.shape[0], max(y_coords) + padding)

                        return frame[y_min:y_max, x_min:x_max]

                    # Crop, resize, and convert left eye and eyebrow region to grayscale
                    left_eye_eyebrow = crop_eye_eyebrow_region(frame, left_eye_eyebrow_points, padding)
                    if left_eye_eyebrow.size != 0:
                        left_eye_eyebrow_resized = cv2.resize(left_eye_eyebrow, (224, 224))
                        #left_eye_eyebrow_gray = cv2.cvtColor(left_eye_eyebrow_resized, cv2.COLOR_BGR2GRAY)
                        left_eye_eyebrow_flipped = cv2.flip(left_eye_eyebrow_resized, 1)  # Flip the left eye image horizontally
                        left_eye_filename = os.path.join(left_eye_output_dir, frame_name)
                        cv2.imwrite(left_eye_filename, left_eye_eyebrow_flipped)
                    else:
                        print(f"Left eye and eyebrow image is empty for frame {frame_name}")

                    # Crop, resize, and convert right eye and eyebrow region to grayscale
                    right_eye_eyebrow = crop_eye_eyebrow_region(frame, right_eye_eyebrow_points, padding)
                    if right_eye_eyebrow.size != 0:
                        right_eye_eyebrow_resized = cv2.resize(right_eye_eyebrow, (224, 224))
                        #right_eye_eyebrow_gray = cv2.cvtColor(right_eye_eyebrow_resized, cv2.COLOR_BGR2GRAY)
                        right_eye_filename = os.path.join(right_eye_output_dir, frame_name)
                        cv2.imwrite(right_eye_filename, right_eye_eyebrow_resized)
                    else:
                        print(f"Right eye and eyebrow image is empty for frame {frame_name}")

    print("Eye and eyebrow extraction complete.")

# Function to process directories and extract eyes and eyebrows
def process_directories(input_dir):
    main_output_dir = "D:\\harsh\\Blepharospasm\\Extracted Colored Eye\\Patient Eyes With Eyebrows_"
    os.makedirs(main_output_dir, exist_ok=True)

    subdirectories = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(input_dir, subdirectory)
        image_files = glob.glob(os.path.join(subdirectory_path, "*.jpg"))
        
        frames = [cv2.imread(img) for img in image_files]
        frame_names = [os.path.basename(img) for img in image_files]

        left_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Left Eye")
        right_eye_output_dir = os.path.join(main_output_dir, subdirectory, "Right Eye")

        extract_eyes_eyebrows_from_frames(frames, left_eye_output_dir, right_eye_output_dir, frame_names)

# Example usage:
input_directory = "D:\\harsh\\Blepharospasm\\Bleph2"
process_directories(input_directory)


### Visualize Mediapipe Landmarks

In [9]:
import mediapipe as mp
import cv2

# Define landmark indices for the eye and eyebrow areas
left_eye_upper_indices = [33,   157,  133]
left_eye_lower_indices = [7,  144, 154]
right_eye_upper_indices = [263,  385,  362]
right_eye_lower_indices = [249,  373, 381]

left_eyebrow_indices = [ 55, 65, 52,63,70]
right_eyebrow_indices = [285,295,282,293,300]

# Function to calculate the midpoint between two landmarks
def calculate_midpoint(lm1, lm2):
    midpoint_x = (lm1.x + lm2.x) / 2
    midpoint_y = (lm1.y + lm2.y) / 2
    return midpoint_x, midpoint_y

# Function to visualize eyebrow points and midpoints of the eyes
def visualize_landmarks(image_path):
    mp_face_mesh = mp.solutions.face_mesh
    mp_drawing = mp.solutions.drawing_utils
    drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    with mp_face_mesh.FaceMesh(static_image_mode=True) as face_mesh:
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Draw eyebrow points
                for idx in left_eyebrow_indices:
                    lm = face_landmarks.landmark[idx]
                    x, y = int(lm.x * image_width), int(lm.y * image_height)
                    cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
                for idx in right_eyebrow_indices:
                    lm = face_landmarks.landmark[idx]
                    x, y = int(lm.x * image_width), int(lm.y * image_height)
                    cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
                
                # Draw midpoints of eyes
                left_eye_midpoints = []
                right_eye_midpoints = []
                
                for i in range(len(left_eye_upper_indices)):
                    left_eye_midpoint = calculate_midpoint(face_landmarks.landmark[left_eye_upper_indices[i]], face_landmarks.landmark[left_eye_lower_indices[i]])
                    left_eye_midpoints.append(left_eye_midpoint)
                    right_eye_midpoint = calculate_midpoint(face_landmarks.landmark[right_eye_upper_indices[i]], face_landmarks.landmark[right_eye_lower_indices[i]])
                    right_eye_midpoints.append(right_eye_midpoint)
                
                for midpoint in left_eye_midpoints:
                    x, y = int(midpoint[0] * image_width), int(midpoint[1] * image_height)
                    cv2.circle(image, (x, y), 2, (0, 0, 255), -1)
                
                for midpoint in right_eye_midpoints:
                    x, y = int(midpoint[0] * image_width), int(midpoint[1] * image_height)
                    cv2.circle(image, (x, y), 2, (0, 0, 255), -1)

    # Display the image
    cv2.imshow('Landmarks Visualization', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function with an example image
image_path = 'C:\\Users\\harsh\\Downloads\\P01-T1-F.jpg'
visualize_landmarks(image_path)


In [39]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

# Define landmark indices
left_eye_upper_indices = [33, 157, 133]
left_eye_lower_indices = [7, 144, 154]
right_eye_upper_indices = [263, 385, 362]
right_eye_lower_indices = [249, 373, 381]
left_eyebrow_indices = [55, 65, 52, 63, 70]
right_eyebrow_indices = [285, 295, 282, 293, 300]

# Mediapipe initialization
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

def calculate_midpoint(lm1, lm2):
    midpoint_x = (lm1.x + lm2.x) / 2
    midpoint_y = (lm1.y + lm2.y) / 2
    return midpoint_x, midpoint_y

def extract_landmark_coords(image_path):
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_image)
    
    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        coords = []
        
        # Add the coordinates for the left eyebrow
        for idx_list in [left_eyebrow_indices]:
            for idx in idx_list:
                lm = face_landmarks.landmark[idx]
                coords.append((lm.x, lm.y))
        
        # Calculate midpoints for the left eye
        left_eye_midpoints = [
            calculate_midpoint(face_landmarks.landmark[left_eye_upper_indices[0]], face_landmarks.landmark[left_eye_lower_indices[0]]),
            calculate_midpoint(face_landmarks.landmark[left_eye_upper_indices[1]], face_landmarks.landmark[left_eye_lower_indices[1]]),
            calculate_midpoint(face_landmarks.landmark[left_eye_upper_indices[2]], face_landmarks.landmark[left_eye_lower_indices[2]])
        ]
        
        # Append left midpoints to the coordinates
        coords.extend(left_eye_midpoints)
        
        # Add the coordinates for the right eyebrow
        for idx_list in [right_eyebrow_indices]:
            for idx in idx_list:
                lm = face_landmarks.landmark[idx]
                coords.append((lm.x, lm.y))
        
        # Calculate midpoints for the right eye
        right_eye_midpoints = [
            calculate_midpoint(face_landmarks.landmark[right_eye_upper_indices[0]], face_landmarks.landmark[right_eye_lower_indices[0]]),
            calculate_midpoint(face_landmarks.landmark[right_eye_upper_indices[1]], face_landmarks.landmark[right_eye_lower_indices[1]]),
            calculate_midpoint(face_landmarks.landmark[right_eye_upper_indices[2]], face_landmarks.landmark[right_eye_lower_indices[2]])
        ]
        
        # Append right midpoints to the coordinates
        coords.extend(right_eye_midpoints)
        
        return coords
    return None

In [55]:
def calculate_features(coords_list):
    coords_array = np.array(coords_list)
    features = []
    
    # Calculate averages and standard deviations separately for x and y coordinates
    x_avgs = []
    y_avgs = []
    x_sds = []
    y_sds = []
    
    # Iterate over each of the 8 points
    for i in range(8):
        x_coords = [coord[0] for coord in coords_array[i]]  # X coordinates for the i-th point
        y_coords = [coord[1] for coord in coords_array[i]]  # Y coordinates for the i-th point
        
        # Calculate average and standard deviation for x and y coordinates
        x_avg = np.mean(x_coords)
        y_avg = np.mean(y_coords)
        x_sd = np.std(x_coords)
        y_sd = np.std(y_coords)
        
        # Append calculated features to the respective lists
        x_avgs.append(x_avg)
        y_avgs.append(y_avg)
        x_sds.append(x_sd)
        y_sds.append(y_sd)
    
    # Extend features in the correct order
    features.extend(x_avgs[:5])  # 5 x averages for the eyebrow
    features.extend(x_sds[:5])   # 5 x standard deviations for the eyebrow
    features.extend(y_avgs[:5])  # 5 y averages for the eyebrow
    features.extend(y_sds[:5])   # 5 y standard deviations for the eyebrow
    features.extend(x_avgs[5:])  # 3 x averages for the eye
    features.extend(x_sds[5:])   # 3 x standard deviations for the eye
    features.extend(y_avgs[5:])  # 3 y averages for the eye
    features.extend(y_sds[5:])   # 3 y standard deviations for the eye

    return features

def process_subfolder(subfolder_path):
    frame_files = [os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.lower().endswith('.jpg') or f.lower().endswith('.png')]
    all_coords_left = [[] for _ in range(8)]
    all_coords_right = [[] for _ in range(8)]
    
    for frame_file in frame_files:
        coords = extract_landmark_coords(frame_file)
        if coords:
            left_eye_coords = coords[:8]  # First 8 points for left side (5 eyebrow + 3 midpoints)
            right_eye_coords = coords[8:]  # Last 8 points for right side (5 eyebrow + 3 midpoints)
            print(f"left_eye_coords: {left_eye_coords}")
            print(f"right_eye_coords: {right_eye_coords}")
            for i in range(8):
                all_coords_left[i].append(left_eye_coords[i])  # Append (x, y) coordinate
                all_coords_right[i].append(right_eye_coords[i])  # Append (x, y) coordinate
    
    
    if not all_coords_left or not all_coords_right:
        return None
    
    all_coords_left = np.array(all_coords_left)
    all_coords_right = np.array(all_coords_right)
    
    left_eye_features = calculate_features(all_coords_left)
    right_eye_features = calculate_features(all_coords_right)
    
    return left_eye_features + right_eye_features

def save_features_to_csv(main_dir, output_csv):
    subfolders = [os.path.join(main_dir, f) for f in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, f))]
    rows = []
    
    for subfolder in subfolders:
        features = process_subfolder(subfolder)
        if features:
            subfolder_name = os.path.basename(subfolder)
            rows.append([subfolder_name] + features + [1])
    
    column_names = ['subfolder_name'] + \
                    [f'left_eyebrow_x{i}_avg' for i in range(1, 6)] + [f'left_eyebrow_x{i}_sd' for i in range(1, 6)] + \
                    [f'left_eyebrow_y{i}_avg' for i in range(1, 6)] + [f'left_eyebrow_y{i}_sd' for i in range(1, 6)] + \
                    [f'left_eye_x{i}_avg' for i in range(1, 4)] + [f'left_eye_x{i}_sd' for i in range(1, 4)] + \
                    [f'left_eye_y{i}_avg' for i in range(1, 4)] + [f'left_eye_y{i}_sd' for i in range(1, 4)] + \
                    [f'right_eyebrow_x{i}_avg' for i in range(1, 6)] + [f'right_eyebrow_x{i}_sd' for i in range(1, 6)] + \
                    [f'right_eyebrow_y{i}_avg' for i in range(1, 6)] + [f'right_eyebrow_y{i}_sd' for i in range(1, 6)] + \
                    [f'right_eye_x{i}_avg' for i in range(1, 4)] + [f'right_eye_x{i}_sd' for i in range(1, 4)] + \
                    [f'right_eye_y{i}_avg' for i in range(1, 4)] + [f'right_eye_y{i}_sd' for i in range(1, 4)] + \
                    ['label']
    
    df = pd.DataFrame(rows, columns=column_names)
    df.to_csv(output_csv, index=False)

# Paths
main_dir = 'D:\\harsh\\Blepharospasm\\Extracted Colored Eye\\New Extracted Frames'
output_csv = 'D:\\harsh\\Blepharospasm\\Extracted Colored Eye\\New Extracted Frames\\features.csv'

# Save features to CSV
save_features_to_csv(main_dir, output_csv)


c:\Users\harsh\anaconda3\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


left_eye_coords: [(0.4666003882884979, 0.395140677690506), (0.44425326585769653, 0.3737052083015442), (0.42690759897232056, 0.3638940453529358), (0.4107111692428589, 0.35072171688079834), (0.40156102180480957, 0.35959184169769287), (0.41721539199352264, 0.40090233087539673), (0.4394477754831314, 0.4094988852739334), (0.4546574503183365, 0.41811105608940125)]
right_eye_coords: [(0.509929358959198, 0.4096401035785675), (0.5371538400650024, 0.40809768438339233), (0.5557963252067566, 0.4130776524543762), (0.573323130607605, 0.41236412525177), (0.5781446099281311, 0.4253934621810913), (0.5526827573776245, 0.4496321380138397), (0.5347472727298737, 0.441473126411438), (0.5135875642299652, 0.4364898353815079)]
left_eye_coords: [(0.5047940611839294, 0.3395877480506897), (0.483786016702652, 0.3236682415008545), (0.46732956171035767, 0.3200291395187378), (0.4507811963558197, 0.31445062160491943), (0.44032567739486694, 0.32548075914382935), (0.4575309008359909, 0.35924962162971497), (0.47803750634

In [1]:
import sys
print(sys.prefix)

c:\Users\harsh\anaconda3


In [1]:
import cv2
import mediapipe as mp

# Define the keypoints indices
left_eye_upper_indices = [33, 157, 133]
left_eye_lower_indices = [7, 144, 154]
right_eye_upper_indices = [263, 385, 362]
right_eye_lower_indices = [249, 373, 381]
left_eyebrow_indices = [55, 65, 52, 63, 70]
right_eyebrow_indices = [285, 295, 282, 293, 300]

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break
    
    # Convert the image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)

    # Draw keypoints if they are detected
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw the keypoints
            for index in left_eye_upper_indices + left_eye_lower_indices + right_eye_upper_indices + right_eye_lower_indices + left_eyebrow_indices + right_eyebrow_indices:
                x = int(face_landmarks.landmark[index].x * image.shape[1])
                y = int(face_landmarks.landmark[index].y * image.shape[0])
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

            # Draw the face mesh (optional, for better visualization)
            mp_drawing.draw_landmarks(image, face_landmarks, 
                                      mp_face_mesh.FACE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                                      mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1))

    # Display the image
    cv2.imshow('MediaPipe FaceMesh', image)
    
    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


c:\Users\harsh\anaconda3\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


AttributeError: module 'mediapipe.python.solutions.face_mesh' has no attribute 'FACE_CONNECTIONS'

: 